# Instalação do MonkAI Agent

Para começar, precisamos instalar o pacote `monkai_agent`. Este framework fornece as classes base e funcionalidades necessárias para criar agentes especializados.

O comando abaixo instalará todas as dependências necessárias:

In [ ]:
#pip install monkai_agent

# Manager e Creator Agents

Este notebook demonstra a implementação de um sistema avançado de agentes usando o padrão Creator e Manager. O exemplo inclui:

1. **PythonDeveloperAgentCreator**: Uma classe especializada para criar agentes desenvolvedores Python
2. **AgentManager**: Gerenciador que coordena os agentes criados
3. **Camada de Segurança**: Implementação de validação de usuário e controle de acesso

Este padrão permite:
- Criar agentes especializados de forma organizada
- Gerenciar múltiplos agentes simultaneamente
- Implementar validações e controles de segurança
- Escalar o sistema de forma modular

In [ ]:
from monkai_agent.monkai_agent_creator import  MonkaiAgentCreator
from monkai_agent.types import Agent
import os

class PythonDeveloperAgentCreator(MonkaiAgentCreator):
    def __init__(self, user):
        super().__init__()
        self.user = user
        self.agent = Agent(
            name="Python Developer Agent",
           instructions="""You are a Python developer and you have to create Python code from text provided by the user.


                1. Interpret the user's text to understand the requirements of the Python code to be generated.
                2. Generate the Python code, defining classes as necessary and following good object-oriented practices. Ensure that the generated code is properly documented.
                3. Check if there is an address provided by the user in the message. If no address is provided, use the current folder.
                4. Create a .py file in the specified folder.
                5. Define a class in the created file with the functionality that meets the conditions specified by the user.
                6. If you cannot provide an answer, trigger the transfer_to_triage function to escalate the request to the triage agent.
            """,
            functions=[  
                        self.verify_address,    
                        self.create_python_file,
                        self.write_code_in_file
                      ])
        

    def verify_address(self, address):
        if not address:
            return os.getcwd()
        if not os.path.isdir(address):
            return f"Endereço {address} não é valido."
        return address

    def create_python_file(self, path, file_name):
        """
        Create a Python file in the specified folder.
        """
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write('')
    
    def write_code_in_file(self,path, file_name, code):
        """
        Write the code in the file.
        """
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write(code)

    def replace_code_in_file(self, path, file_name, code):
        """
        Replace the code in the file.
        """
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write(code)

    def get_agent(self) -> Agent:
        """
        Returns the Python Developer Agent.
        """
        return self.agent
    
    def get_agent_briefing(self) -> str:
        return "You are a Python developer and you have to create Python code from text provided by the user."

            

# Implementação do PythonDeveloperAgentCreator

Esta seção implementa a classe `PythonDeveloperAgentCreator`, que herda de `MonkaiAgentCreator`. 
Esta classe é responsável por criar agentes especializados em desenvolvimento Python.

### Principais Componentes:

1. **Inicialização**:
   - Recebe um usuário como parâmetro
   - Configura o agente com instruções específicas

2. **Funções do Agente**:
   - `verify_address`: Valida e normaliza endereços de diretório
   - `create_python_file`: Cria arquivos Python vazios
   - `write_code_in_file`: Escreve código em arquivos
   - `replace_code_in_file`: Substitui conteúdo de arquivos existentes

3. **Instruções do Agente**:
   O agente é configurado para:
   - Interpretar requisitos do usuário
   - Gerar código Python seguindo boas práticas
   - Criar e manipular arquivos
   - Escalar para triagem quando necessário

In [ ]:
from monkai_agent.base import AgentManager
from monkai_agent.repl import run_demo_loop
from openai import AzureOpenAI
import config

agents_creators = []
agents_creators.append(PythonDeveloperAgentCreator(user ='valid_user'))
client=AzureOpenAI(
            api_key=config.OPENAI_API_KEY_BRASILSOUTH,
            api_version=config.GPT4o_OPENAI_API_VERSION_BRASILSOUTH,
            azure_endpoint=config.OPENAI_AZURE_ENDPOINT_BRASILSOUTH,
        )
agent_manager = AgentManager(client=client, agents_creators=agents_creators)
await run_demo_loop(agent_manager, model=config.GPT4o_OPENAI_GPT_MODEL_BRASILSOUTH)

# Configuração e Execução do AgentManager

Esta seção demonstra como configurar e executar o sistema de agentes. O processo envolve:

1. **Criação dos Agentes**:
   - Instanciação do PythonDeveloperAgentCreator
   - Configuração com usuário válido

2. **Configuração do Cliente Azure OpenAI**:
   - Configuração das credenciais
   - Definição do endpoint e versão da API
   - Especificação do modelo GPT

3. **Inicialização do AgentManager**:
   - Integração dos agentes criados
   - Configuração do cliente OpenAI
   - Execução do loop de demonstração

**Importante**: Substitua as variáveis de configuração pelos seus próprios valores de API e endpoint.

# Camada de Segurança

Esta seção implementa uma versão aprimorada do `PythonDeveloperAgentCreator` com recursos de segurança adicionados.

### Principais Melhorias:

1. **Validação de Usuário**:
   - Implementação do método `is_user_valid`
   - Verificação de usuários autorizados

2. **Decorador de Validação**:
   - Uso do decorador `@validate`
   - Proteção de métodos críticos

3. **Métodos Protegidos**:
   - `create_python_file`: Requer validação
   - `write_code_in_file`: Requer validação
   - Controle granular de acesso

Esta implementação demonstra como adicionar uma camada de segurança robusta ao sistema de agentes.

In [ ]:
from monkai_agent.monkai_agent_creator import  MonkaiAgentCreator
from monkai_agent.types import Agent
from monkai_agent.security import validate
import os

class PythonDeveloperAgentCreator(MonkaiAgentCreator):
    def __init__(self, user):
        super().__init__()
        self.user = user
        self.agent = Agent(
            name="Python Developer Agent",
           instructions="""You are a Python developer and you have to create Python code from text provided by the user.


                1. Interpret the user's text to understand the requirements of the Python code to be generated.
                2. Generate the Python code, defining classes as necessary and following good object-oriented practices. Ensure that the generated code is properly documented.
                3. Check if there is an address provided by the user in the message. If no address is provided, use the current folder.
                4. Create a .py file in the specified folder.
                5. Define a class in the created file with the functionality that meets the conditions specified by the user.
                6. If you cannot provide an answer, trigger the transfer_to_triage function to escalate the request to the triage agent.
            """,
            functions=[  
                        self.verify_address,    
                        self.create_python_file,
                        self.write_code_in_file
                      ])
        
    def is_user_valid(self):
        if self.user=="valid_user":
            return True
        else:
            return False
    
    
    def verify_address(self, address):
        if not address:
            return os.getcwd()
        if not os.path.isdir(address):
            return f"Endereço {address} não é valido."
        return address

    @validate(is_user_valid)
    def create_python_file(self, path, file_name):
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write('')
    
    
    @validate(is_user_valid)
    def write_code_in_file(self,path, file_name, code):
        """
        Write the code in the file.
        """
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write(code)

    def replace_code_in_file(self, path, file_name, code):
        """
        Replace the code in the file.
        """
        complete_path = os.path.join(path, file_name)
        with open(complete_path, 'w') as f:
            f.write(code)

    def get_agent(self) -> Agent:
        """
        Returns the Python Developer Agent.
        """
        return self.agent
    
    def get_agent_briefing(self) -> str:
        return "You are a Python developer and you have to create Python code from text provided by the user."

            

# Implementação com Segurança

Abaixo está a implementação completa do `PythonDeveloperAgentCreator` com a camada de segurança integrada.
Esta versão inclui:

1. **Sistema de Validação**:
   ```python
   @validate(is_user_valid)  # Decorador que verifica a validade do usuário
   def metodo_protegido(self):
       # Código protegido aqui
   ```

2. **Controle de Acesso**:
   - Validação antes de operações de arquivo
   - Verificação de usuário em métodos críticos
   - Proteção contra acessos não autorizados

3. **Boas Práticas**:
   - Documentação clara dos métodos
   - Separação de responsabilidades
   - Manipulação segura de arquivos

# Teste com Usuário Inválido

Esta seção demonstra o comportamento do sistema quando um usuário não autorizado tenta acessá-lo.

### Pontos Importantes:

1. **Configuração de Teste**:
   - Uso de um usuário não autorizado ('jose angel')
   - Mesmo ambiente e configurações anteriores

2. **Comportamento Esperado**:
   - As operações protegidas serão bloqueadas
   - O sistema registrará as tentativas de acesso
   - Mensagens de erro apropriadas serão geradas

3. **Propósito**:
   - Validar o funcionamento da camada de segurança
   - Demonstrar o controle de acesso em ação
   - Verificar a robustez do sistema

In [ ]:
from monkai_agent.base import AgentManager
from monkai_agent.repl import run_demo_loop

agents_creators = []
agents_creators.append(PythonDeveloperAgentCreator(user ='jose angel'))
agent_manager = AgentManager(agents_creators=agents_creators, api_key='api-key')
await run_demo_loop(agent_manager, model=config.GPT4o_OPENAI_GPT_MODEL_BRASILSOUTH)